In [8]:
import simpy
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
import random

class Consumer:
    def __init__(self, env, queue, delay, busy_time):
        self.env = env
        self.queue = queue
        self.state = 'pending'
        self.action = env.process(self.pending())
        self.delay = delay
        self.busy_time = busy_time

    def pending(self):
        self.queue.pending.append(self)
        yield self.env.timeout(self.delay * random.uniform(1, 1.2))
        self.queue.pending.remove(self)
        self.queue.idle.append(self)
        self.action = self.env.process(self.work())

    def work(self):
        while True:
            # Wait for work
            yield self.queue.store.get()
            self.queue.idle.remove(self)
            self.queue.processing.append(self)
            yield self.env.timeout(self.busy_time * random.gauss(1, 0.1))
            # Finish work and terminate
            self.queue.processing.remove(self)
            break

class WorkQueue:
    def __init__(self, env, min_idle_consumers, delay, busy_time):
        self.env = env
        self.idle = []
        self.processing = []
        self.pending = []
        self.store = simpy.Store(env)
        self.delay = delay
        self.busy_time = busy_time
        self.min_idle_consumers = min_idle_consumers

        self.setup_initial_consumers()

    def setup_initial_consumers(self):
        for _ in range(self.min_idle_consumers):
            self.add_pending_consumer()

    def add_pending_consumer(self):
        Consumer(self.env, self, self.delay, self.busy_time)

    def add_work(self, count):
        for _ in range(count):
            self.store.put("work")

# Function to run the simulation with adjustable parameters
def run_simulation(min_idle_consumers, consumer_startup_delay, consumer_busy_time, max_consumers,
                   work_burst_size, consumer_start_percentage, burst_count, check_interval, total_runtime=3600):
    # Data storage for plotting
    time_data = []
    idle_consumers_data = []
    busy_consumers_data = []
    pending_consumers_data = []

    def generate_work(env, queue):
        while True:
            yield env.timeout(total_runtime/burst_count * random.uniform(0.1, 2.5))
            queue.add_work(work_burst_size)

    def poll_idle_consumers(env, queue):
        while True:
            yield env.timeout(check_interval)
            total_consumers = len(queue.idle) + len(queue.processing)
            if len(queue.idle) < min_idle_consumers:
                consumers_to_start = max(int(total_consumers * consumer_start_percentage), 1)
                # start up to the max consumers, no more.
                consumers_to_start = min(max_consumers - total_consumers, consumers_to_start)
                for _ in range(consumers_to_start):
                    queue.add_pending_consumer()

    def monitor_consumers(env, queue):
        while True:
            yield env.timeout(1)  # Collect data at every time step
            time_data.append(env.now)
            idle_consumers_data.append(len(queue.idle))
            busy_consumers_data.append(len(queue.processing))
            pending_consumers_data.append(len(queue.pending))

    # Setup simulation environment
    env = simpy.Environment()
    queue = WorkQueue(env, min_idle_consumers, consumer_startup_delay, consumer_busy_time)

    # Start processes
    env.process(generate_work(env, queue))
    env.process(poll_idle_consumers(env, queue))
    env.process(monitor_consumers(env, queue))

    env.run(until=total_runtime)

    plt.clf()
    plt.close('all')

    plt.figure(figsize=(15, 5))
    plt.stackplot(time_data, idle_consumers_data, busy_consumers_data, labels=['Idle Consumers', 'Busy Consumers'])
    plt.legend(loc='upper right')
    plt.xlabel('Time')
    plt.ylabel('Number of Consumers')
    plt.title('Idle and Busy Consumers Over Time')
    plt.show()

    plt.figure(figsize=(15, 5))
    plt.plot(time_data, pending_consumers_data, label='Pending Consumers')
    plt.legend(loc='upper right')
    plt.xlabel('Time (minutes)')
    plt.ylabel('Number of Pending Consumers')
    plt.title('Pending Consumers Over Time')
    plt.show()

# Interactive sliders for parameters
interact(run_simulation,
         min_idle_consumers=widgets.IntSlider(value=40, min=0, max=100, step=1, description='Min Idle Consumers'),
         max_consumers=widgets.IntSlider(value=500, min=0, max=1000, step=1, description='Max Consumers'),
         consumer_startup_delay=widgets.IntSlider(value=200, min=60, max=300, step=30, description='Startup Delay (s)'),
         consumer_busy_time=widgets.IntSlider(value=600, min=1, max=3600, step=60, description='Busy Time (s)'),
         work_burst_size=widgets.IntSlider(value=80, min=1, max=200, step=1, description='Burst Size'),
         consumer_start_percentage=widgets.FloatSlider(value=0.50, min=0.01, max=2, step=0.01, description='Start %'),
         burst_count=widgets.IntSlider(value=48, min=0, max=100, step=1, description='bursts'),
         check_interval=widgets.IntSlider(value=100, min=1, max=1200, step=1, description='Check Interval (s)'),
         total_runtime=widgets.IntSlider(value=3600 * 4, min=60, max=86400, step=3600, description='Total Runtime (s)')
        )

interactive(children=(IntSlider(value=40, description='Min Idle Consumers'), IntSlider(value=200, description=…

<function __main__.run_simulation(min_idle_consumers, consumer_startup_delay, consumer_busy_time, max_consumers, work_burst_size, consumer_start_percentage, burst_count, check_interval, total_runtime=3600)>